# Pendahuluan
Penelitian ini bertujuan untuk melakukan analisis sentimen pada data ulasan dari Google Place Review untuk Perguruan Tinggi. Dengan analisis sentimen ini, penulis dapat mengidentifikasi pandangan umum pengguna terkait layanan atau aspek tertentu yang terdapat pada universitas. Terdapat dua dimensi sentimen yang dianalisis, yaitu:

1. Sentimen: Sentimen yang merefleksikan emosi pengguna secara keseluruhan terhadap layanan, seperti "positif," "negatif," dan "netral."
2. Aspek: Kategori yang menunjukkan aspek tertentu dari ulasan yang diberikan, seperti "akademik," "fasilitas," "administrasi," dan "lainnya."

## Persiapan Data

Analisis ini dibagi menjadi dua bagian, yaitu data yang telah diberi label secara manual oleh penulis (data-labeled) dan data yang belum memiliki label (data-unlabeled). Data berlabel ini digunakan untuk melatih model prediktif, yang kemudian akan diaplikasikan pada data tidak berlabel untuk mengotomatisasi proses penentuan sentimen.

Datanya sebagai berikut:
/content/data-hendri.csv
/content/data-hendri-labeled.csv
/content/data-hendri-nonlabeled.csv

In [ ]:
import pandas as pd

# Load dataset dari path yang diberikan
data_awal = pd.read_csv('/content/data-hendri.csv', delimiter=';')
data_labeled = pd.read_csv('/content/data-hendri-labeled.csv', delimiter=';')
data_unlabeled = pd.read_csv('/content/data-hendri-nonlabeled.csv', delimiter=';')


# Menampilkan beberapa baris pertama dari dataset
data.head()
print("Data Awal")
display(data_awal.head())

# Menampilkan beberapa baris pertama dari masing-masing dataset untuk verifikasi
print("Data Berlabel:")
display(data_labeled.head())

print("Data Tidak Berlabel:")
display(data_unlabeled.head())


Data Awal


,Review
0,Tempat Kuliah Orang Berdasi
1,Unggul dalam trend teknologi informasi
2,Kampus ungu
3,Kampusku Tercinta
4,Tempat kuliah orang berdasi.\n#entreprenur


Data Berlabel:


,No,Review,sentimen,aspek
0,1,Tempat Kuliah Orang Berdasi,netral,fasilitas
1,2,Unggul dalam trend teknologi informasi,positif,akademik
2,3,Kampus ungu,netral,umum
3,4,Kampusku Tercinta,positif,umum
4,5,Tempat kuliah orang berdasi.\n#entreprenur,netral,fasilitas


Data Tidak Berlabel:


,No,Review,sentimen,aspek
0,201,Lab komputer nya menyediakan perangkat kompute...,NaN,NaN
1,202,"Staff dan Humas nya ramah ramah, ada unit bisn...",NaN,NaN
2,203,Kampus yg luar biasa dosen2 yg bagus2 kualitasnya,NaN,NaN
3,204,"Nyonya alumsi sini, kampus ini berkembang pesat",NaN,NaN
4,205,Kampus nya orang berdasi... Bangga jadi lulusa...,NaN,NaN


### Penjelasan

Setelah memuat dataset, penulis menampilkan data awal, data berlabel, dan data tidak berlabel sebagai berikut:

1. Data Awal (data_awal): Dataset ini memuat data ulasan awal yang diperoleh dari Google Place Review, yang kemudian dibagi menjadi dua bagian untuk keperluan pelabelan dan pemrosesan lebih lanjut.

2. Data Berlabel (data_labeled): Dataset ini memuat 200 contoh ulasan yang telah dilabeli secara manual, dengan dua kolom utama, yaitu sentimen untuk emosi keseluruhan dan aspek untuk kategori ulasan. Data ini akan digunakan sebagai data pelatihan untuk membangun model analisis sentimen.

3. Data Tidak Berlabel (data_unlabeled): Dataset ini berisi sisa data (ulasan 201-600) yang belum diberi label. Data ini akan digunakan untuk menguji performa model dalam melakukan prediksi sentimen secara otomatis setelah model dilatih.

Melalui proses ini, penulis dapat melakukan analisis lebih mendalam terhadap data ulasan dan memahami bagaimana persepsi pengguna terhadap berbagai aspek di data review dalam hal ini adalah review dari Universitas AMIKOM Yogyakarta. Langkah-langkah berikutnya akan mencakup preprocessing dan pelatihan model untuk mengklasifikasikan sentimen dan kategori secara otomatis.

In [ ]:
# Fitur dan label untuk data berlabel
X_train = tokens_labeled['input_ids']
y_train = data_labeled['sentimen_encoded']  # Gunakan kolom encoded hasil mapping


In [ ]:
# Inisialisasi dan latih model SVM
model = SVC()
model.fit(X_train, y_train)


SVC()

In [ ]:
# Fitur untuk data tidak berlabel
X_unlabeled = tokens_unlabeled['input_ids']

# Prediksi sentimen pada data tidak berlabel
y_pred_unlabeled = model.predict(X_unlabeled)

# Tambahkan hasil prediksi ke kolom 'sentimen_prediksi' di data tidak berlabel
data_unlabeled['sentimen_prediksi'] = y_pred_unlabeled


In [ ]:
# Simpan data tidak berlabel dengan hasil prediksi
data_unlabeled.to_csv('/content/data-hendri-predicted.csv', index=False)


# Hasil Dan Pembahasan

# HASIL

# Pre Processing Data

## 1. Data Cleaning
Deskripsi Awal:
Langkah pertama dalam preprocessing adalah Data Cleaning. Penulis akan membersihkan teks dari karakter yang tidak diperlukan, seperti URL, tanda baca, dan angka, yang dapat mengganggu pemrosesan teks dan mempengaruhi akurasi model.

In [ ]:
import re

# Fungsi untuk membersihkan teks dari URL, angka, tanda baca, emoticon, baris baru, dan simbol #
def clean_text(text):
    # Hapus semua URL yang diawali dengan http, https, atau www
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    # Hapus simbol # dan baris baru
    text = re.sub(r'[\n#]', ' ', text)
    # Hapus emoticon dan simbol emoji menggunakan regex karakter unicode
    text = re.sub(r'[^\w\s]', '', text, flags=re.UNICODE)
    # Hapus angka dan karakter non-alfabet selain spasi
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Konversi ke lowercase dan hilangkan spasi berlebih
    text = text.lower().strip()
    # Hapus spasi berlebih di antara kata-kata
    text = re.sub(r'\s+', ' ', text)
    return text

# Terapkan data cleaning pada kolom 'Review'
data_labeled['cleaned_review'] = data_labeled['Review'].apply(clean_text)
data_unlabeled['cleaned_review'] = data_unlabeled['Review'].apply(clean_text)

# Tampilkan hasil beberapa baris pertama untuk verifikasi
print(data_labeled[['Review', 'cleaned_review']].head())
print(data_unlabeled[['Review', 'cleaned_review']].head())

                                       Review  \
0                 Tempat Kuliah Orang Berdasi   
1      Unggul dalam trend teknologi informasi   
2                                 Kampus ungu   
3                           Kampusku Tercinta   
4  Tempat kuliah orang berdasi.\n#entreprenur   

                            cleaned_review  
0              tempat kuliah orang berdasi  
1   unggul dalam trend teknologi informasi  
2                              kampus ungu  
3                        kampusku tercinta  
4  tempat kuliah orang berdasi entreprenur  
                                              Review  \
0  Lab komputer nya menyediakan perangkat kompute...   
1  Staff dan Humas nya ramah ramah, ada unit bisn...   
2  Kampus yg luar biasa dosen2 yg bagus2 kualitasnya   
3    Nyonya alumsi sini, kampus ini berkembang pesat   
4  Kampus nya orang berdasi... Bangga jadi lulusa...   

                                      cleaned_review  
0  lab komputer nya menyediakan perangkat k

### Penjelasan
Hasil dari script ini adalah teks pada kolom Review yang telah dibersihkan, disimpan dalam kolom cleaned_review. Penulis dapat melihat teks yang lebih terstruktur tanpa adanya URL, tanda baca yang tidak relevan, atau angka, yang akan memudahkan analisis lanjutan.



## 2. Normalisasi
Deskripsi Awal:
Setelah Data Cleaning, penulis akan melakukan Normalisasi untuk mengganti kata-kata yang tidak baku atau slang dengan kata-kata yang lebih formal. Hal ini membantu model dalam memahami makna kata-kata tersebut dengan lebih baik.

In [ ]:
# Kamus normalisasi sederhana (dapat ditambah sesuai kebutuhan)
normalization_dict = {
    "gak": "tidak",
    "nggak": "tidak",
    "klo": "kalau",
    "aja": "saja",
    "nih": "ini",
    "neh": "ini",
    "deh": "sudah",
    "wkwk": "haha",
    "wkwkwk": "haha",
    "wkwkwkwkwk": "haha",
    "wkwkwkwk":"haha"
}

# Fungsi untuk melakukan normalisasi teks
def normalize_text(text):
    words = text.split()
    normalized_words = [normalization_dict.get(word, word) for word in words]
    return " ".join(normalized_words)

# Terapkan normalisasi pada kolom 'cleaned_review'
data_labeled['cleaned_review'] = data_labeled['cleaned_review'].apply(normalize_text)
data_unlabeled['cleaned_review'] = data_unlabeled['cleaned_review'].apply(normalize_text)

# Tampilkan hasil beberapa baris pertama untuk verifikasi
print(data_labeled[['Review', 'cleaned_review']].head())


                                       Review  \
0                 Tempat Kuliah Orang Berdasi   
1      Unggul dalam trend teknologi informasi   
2                                 Kampus ungu   
3                           Kampusku Tercinta   
4  Tempat kuliah orang berdasi.\n#entreprenur   

                            cleaned_review  
0              tempat kuliah orang berdasi  
1   unggul dalam trend teknologi informasi  
2                              kampus ungu  
3                        kampusku tercinta  
4  tempat kuliah orang berdasi entreprenur  


### Penjelasan Output:
Script ini menggantikan kata-kata slang dengan padanan kata formal sesuai dengan normalization_dict yang telah ditentukan. Penulis akan mendapatkan teks yang lebih baku, sehingga model dapat memahami dan menggeneralisasi konteks dengan lebih baik.



## Stemming

Deskripsi Awal:
Langkah selanjutnya adalah Stemming, di mana penulis akan mengembalikan kata-kata ke bentuk dasarnya. Dengan langkah ini, variasi kata yang memiliki arti sama akan diseragamkan, sehingga model dapat lebih fokus pada arti dasar dari kata tersebut.

In [ ]:
!pip install Sastrawi

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Inisialisasi stemmer Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Daftar pengecualian kata-kata yang tidak boleh di-stem
exception_words = {"dasi"}  # Contoh daftar kata

# Fungsi stemming dengan pengecualian
def stem_with_exceptions(text):
    words = text.split()
    stemmed_words = [word if word in exception_words else stemmer.stem(word) for word in words]
    return " ".join(stemmed_words)

# Terapkan stemming pada kolom 'cleaned_review' dengan pengecualian
data_labeled['cleaned_review'] = data_labeled['cleaned_review'].apply(stem_with_exceptions)
data_unlabeled['cleaned_review'] = data_unlabeled['cleaned_review'].apply(stem_with_exceptions)

# Tampilkan hasil beberapa baris pertama untuk verifikasi
print(data_labeled[['Review', 'cleaned_review']].head())


                                       Review  \
0                 Tempat Kuliah Orang Berdasi   
1      Unggul dalam trend teknologi informasi   
2                                 Kampus ungu   
3                           Kampusku Tercinta   
4  Tempat kuliah orang berdasi.\n#entreprenur   

                        cleaned_review  
0              tempat kuliah orang das  
1     unggul trend teknologi informasi  
2                          kampus ungu  
3                         kampus cinta  
4  tempat kuliah orang das entreprenur  


### Penjelasan Output:
Setelah proses stemming, setiap kata dalam teks telah dikembalikan ke bentuk dasarnya. Hal ini memudahkan penulis untuk menghilangkan perbedaan akibat variasi kata tanpa mengubah makna dasar, misalnya “makanannya” menjadi “makan.”

## 4. Stop Word Removal
Deskripsi Awal:
Setelah stemming, penulis akan melakukan Stop Word Removal untuk menghapus kata-kata yang umum tetapi tidak menambah nilai pada analisis sentimen atau aspek. Penghapusan stopwords memungkinkan model untuk lebih fokus pada kata-kata penting dalam setiap kalimat.

In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Inisialisasi stopwords Sastrawi
stop_factory = StopWordRemoverFactory()
stop_words = set(stop_factory.get_stop_words())

# Fungsi untuk menghapus stopwords
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return " ".join(filtered_words)

# Terapkan stopword removal pada kolom 'cleaned_review'
data_labeled['cleaned_review'] = data_labeled['cleaned_review'].apply(remove_stopwords)
data_unlabeled['cleaned_review'] = data_unlabeled['cleaned_review'].apply(remove_stopwords)

# Tampilkan hasil beberapa baris pertama untuk verifikasi
print(data_labeled[['Review', 'cleaned_review']].head())


                                       Review  \
0                 Tempat Kuliah Orang Berdasi   
1      Unggul dalam trend teknologi informasi   
2                                 Kampus ungu   
3                           Kampusku Tercinta   
4  Tempat kuliah orang berdasi.\n#entreprenur   

                        cleaned_review  
0              tempat kuliah orang das  
1     unggul trend teknologi informasi  
2                          kampus ungu  
3                         kampus cinta  
4  tempat kuliah orang das entreprenur  


### Penjelasan Output:
Stopwords yang umum dan tidak memberikan makna spesifik pada teks telah dihapus. Penulis kini mendapatkan teks yang lebih fokus pada kata-kata bermakna, memungkinkan model untuk belajar lebih baik dalam memahami konteks kalimat.

## 5. Encoding
Deskripsi Awal:
Penulis akan melakukan Encoding untuk mengonversi nilai kategori pada kolom sentimen dan aspek menjadi skala numerik, sehingga dapat digunakan dalam model.


In [ ]:
# Mapping skala nominal untuk kolom 'sentimen' dan 'aspek'
sentimen_mapping = {'positif': 1, 'negatif': 2, 'netral': 3}
aspek_mapping = {'akademik': 1, 'fasilitas': 2, 'administrasi': 3, 'umum': 4}

# Terapkan mapping pada data berlabel
data_labeled['sentimen_encoded'] = data_labeled['sentimen'].map(sentimen_mapping)
data_labeled['aspek_encoded'] = data_labeled['aspek'].map(aspek_mapping)

# Tampilkan hasil beberapa baris pertama untuk verifikasi
print(data_labeled[['sentimen', 'sentimen_encoded', 'aspek', 'aspek_encoded']].head())


  sentimen  sentimen_encoded      aspek  aspek_encoded
0   netral                 3  fasilitas              2
1  positif                 1   akademik              1
2   netral                 3       umum              4
3  positif                 1       umum              4
4   netral                 3  fasilitas              2


### Penjelasan Output:
Kolom sentimen dan aspek kini telah dikonversi menjadi nilai numerik sesuai skala nominal yang ditetapkan. Penulis dapat menggunakan data ini sebagai target label dalam pelatihan model.

## 6. Tokenizing
Deskripsi Awal:
Langkah terakhir dalam preprocessing adalah Tokenizing. Penulis akan memisahkan teks ke dalam token-token, seperti kata-kata atau frasa, agar teks dapat diolah menjadi fitur untuk model.

In [ ]:
from transformers import AutoTokenizer

# Load tokenizer IndoBERT
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")

# Tokenisasi teks pada kolom 'cleaned_review'
tokens_labeled = tokenizer(
    data_labeled['cleaned_review'].tolist(),
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

tokens_unlabeled = tokenizer(
    data_unlabeled['cleaned_review'].tolist(),
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

# Tampilkan bentuk tensor untuk verifikasi
print(tokens_labeled['input_ids'][:2])


### Penjelasan Output:
Teks telah ditokenisasi menjadi representasi berbentuk tensor. Penulis kini memiliki data siap pakai untuk feature extraction pada model IndoBERT, yang akan menjadi input bagi model SVM atau model lain yang dipilih.



# Data Modelling

Untuk melaksanakan percobaan ini, penulis akan memisahkan data labeled menjadi data training dan testing menggunakan kombinasi yang diinginkan (70:30, 80:20, dan 90:10). Penulis kemudian akan melakukan prediksi dengan beberapa metode, yaitu SVM, SVM dengan Particle Swarm Optimization (PSO), dan SVM-PSO dengan fitur IndoBERT. Setiap model akan dievaluasi berdasarkan akurasi, F1-score, dan metrik lain, lalu dibandingkan dengan input manual penulis.

Berikut ini adalah skenario script yang akan berjalan bertahap dengan penjelasan per bagian:

In [ ]:
# prompt: show head of data-hendri-labeled.csv

import pandas as pd
data_labeled = pd.read_csv('/content/data-hendri-labeled.csv', delimiter=';')
print(data_labeled.head())

   No                                      Review sentimen      aspek
0   1                 Tempat Kuliah Orang Berdasi   netral  fasilitas
1   2      Unggul dalam trend teknologi informasi  positif   akademik
2   3                                 Kampus ungu   netral       umum
3   4                           Kampusku Tercinta  positif       umum
4   5  Tempat kuliah orang berdasi.\n#entreprenur   netral  fasilitas


## 1. Pembagian Data Training dan Testing
Deskripsi Awal:
Penulis akan membagi data labeled menjadi data training dan testing menggunakan pembagian 70:30, 80:20, dan 90:10, sehingga penulis dapat menguji performa model dengan kombinasi data yang berbeda.


In [ ]:
from sklearn.model_selection import train_test_split

# Daftar kombinasi data train-test
splits = [(0.7, 0.3), (0.8, 0.2), (0.9, 0.1)]
datasets = {}

# Membagi data untuk masing-masing kombinasi dengan presisi
for train_size, test_size in splits:
    train_data, test_data = train_test_split(data_labeled, test_size=test_size, random_state=42)
    datasets[f"{int(train_size * 100)}:{int(test_size * 100)}"] = (train_data, test_data)


### Penjelasan Output:
Script ini menghasilkan tiga set data training dan testing dengan pembagian 70:30, 80:20, dan 90:10, yang disimpan dalam datasets. Setiap pembagian akan digunakan untuk eksperimen model secara terpisah.



##2. Modelling dengan SVM
Deskripsi Awal:
Penulis akan melatih model SVM pada setiap kombinasi data (70:30, 80:20, dan 90:10) untuk memprediksi kolom sentimen dan aspek, lalu mengevaluasi hasilnya.

## Dengan Representasi TF-IDF
Deskripsi Awal:
Penulis akan mengubah teks menjadi representasi fitur numerik menggunakan TF-IDF sebelum memprosesnya dengan model SVM. TF-IDF mengukur seberapa penting suatu kata dalam teks, berdasarkan frekuensi kemunculannya dan signifikansi dalam seluruh korpus.

### Penjelasan Output:
Script ini pertama-tama mentransformasi kolom teks cleaned_review ke bentuk numerik dengan TF-IDF sebelum melatih model SVM. Dengan TF-IDF, setiap kata diubah menjadi representasi numerik berdasarkan frekuensi kemunculannya, sehingga model SVM dapat menerima input ini tanpa error.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Inisialisasi TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Fungsi untuk melatih dan menguji model SVM menggunakan TF-IDF
def train_and_evaluate_svm(train_data, test_data, target_column):
    # Transformasi teks ke TF-IDF
    X_train = tfidf_vectorizer.fit_transform(train_data['cleaned_review'])
    y_train = train_data[target_column]
    X_test = tfidf_vectorizer.transform(test_data['cleaned_review'])
    y_test = test_data[target_column]

    # Inisialisasi dan pelatihan model SVM
    model = SVC()
    model.fit(X_train, y_train)

    # Prediksi dan evaluasi
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    return accuracy, report

# Melakukan eksperimen SVM untuk setiap kombinasi data dan target (sentimen & aspek)
for split, (train_data, test_data) in datasets.items():
    print(f"\nEvaluasi SVM pada pembagian data {split} untuk prediksi Sentimen:")
    accuracy, report = train_and_evaluate_svm(train_data, test_data, 'sentimen_encoded')
    print(f"Akurasi: {accuracy}\n{report}")

    print(f"\nEvaluasi SVM pada pembagian data {split} untuk prediksi Aspek:")
    accuracy, report = train_and_evaluate_svm(train_data, test_data, 'aspek_encoded')
    print(f"Akurasi: {accuracy}\n{report}")



Evaluasi SVM pada pembagian data 70:30 untuk prediksi Sentimen:
Akurasi: 0.7377049180327869
              precision    recall  f1-score   support

           1       0.78      0.91      0.84        44
           2       0.00      0.00      0.00         1
           3       0.50      0.31      0.38        16

    accuracy                           0.74        61
   macro avg       0.43      0.41      0.41        61
weighted avg       0.70      0.74      0.71        61


Evaluasi SVM pada pembagian data 70:30 untuk prediksi Aspek:
Akurasi: 0.6065573770491803
              precision    recall  f1-score   support

           1       0.67      0.18      0.29        11
           2       0.80      0.36      0.50        22
           4       0.56      0.96      0.71        28

    accuracy                           0.61        61
   macro avg       0.68      0.50      0.50        61
weighted avg       0.67      0.61      0.56        61


Evaluasi SVM pada pembagian data 80:20 untuk prediksi 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Interpretasi

Interpretasi hasil evaluasi di atas mencakup analisis akurasi serta metrik evaluasi lainnya (presisi, recall, dan F1-score) untuk model SVM dengan representasi TF-IDF dalam memprediksi sentimen dan aspek pada setiap kombinasi pembagian data (70:30, 80:20, dan 90:10). Berikut penjelasan per bagian:

### Evaluasi SVM pada Pembagian Data 70:30
#### Prediksi Sentimen

1. **Akurasi**: 0.7377 (sekitar 74%) – artinya, model berhasil memprediksi sentimen dengan benar untuk sekitar 74% dari data uji.
2. **Kelas 1 (Positif)**: Memiliki presisi tinggi (0.78) dan recall tinggi (0.91), menunjukkan bahwa model mampu mendeteksi kelas "positif" secara andal dan mendeteksi sebagian besar contoh positif yang ada dalam data uji.
3. **Kelas 2 (Negatif)**: Presisi dan recall sama-sama 0.0 karena hanya terdapat satu contoh di kelas ini, dan model tidak berhasil memprediksinya dengan benar.
4. **Kelas 3 (Netral)**: Presisi (0.50) dan recall (0.31) menunjukkan model mengalami kesulitan dalam mengenali kelas ini, sehingga banyak contoh netral yang terlewat.
5. **Macro Average dan Weighted Average**: Macro average menunjukkan kinerja keseluruhan per kelas secara rata-rata, sementara weighted average menimbang kinerja berdasarkan jumlah contoh per kelas.

#### Prediksi Aspek

1. **Akurasi**: 0.6066 (sekitar 61%) – model memprediksi aspek dengan benar untuk sekitar 61% data uji.
2. **Kelas 1 (Akademik)**: Presisi cukup baik (0.67), tetapi recall rendah (0.18), artinya model cenderung memberikan hasil positif palsu untuk kelas ini.
3. **Kelas 2 (Fasilitas)**: Memiliki presisi (0.80) dan recall (0.36) yang menunjukkan adanya bias prediksi kelas "fasilitas".
4. **Kelas 4 (Umum)**: Model cenderung mengklasifikasikan ke kelas ini dengan recall tinggi (0.96), tetapi presisi cukup rendah (0.56), menunjukkan kecenderungan model mengklasifikasikan ke kelas "umum".

### Evaluasi SVM pada Pembagian Data 80:20

#### Prediksi Sentimen

1. **Akurasi**: 0.7561 (sekitar 76%) – performa model sedikit meningkat dibandingkan pembagian 70:30.
2. **Kelas 1 (Positif)**: Tingkat presisi (0.75) dan recall (0.96) tinggi, menunjukkan kelas "positif" dikenali dengan baik.
3. **Kelas 3 (Netral)**: Presisi tinggi (0.80) tetapi recall rendah (0.31), menunjukkan adanya prediksi positif palsu.

#### Prediksi Aspek

1. **Akurasi**: 0.5854 (sekitar 59%) – performa menurun sedikit dibandingkan pembagian 70:30.
2. **Kelas 1 (Akademik)**: Presisi rendah (0.50) dan recall sangat rendah (0.12), menunjukkan banyak kesalahan pada kelas ini.
3. **Kelas 2 (Fasilitas)**: Memiliki presisi yang baik (0.75), tetapi recall masih rendah (0.40).
4. **Kelas 4 (Umum)**: Recall tinggi (0.94) menunjukkan kecenderungan model untuk mengklasifikasikan data ke kelas "umum".

### Evaluasi SVM pada Pembagian Data 90:10

#### Prediksi Sentimen

1. **Akurasi**: 0.7143 (sekitar 71%) – performa sedikit menurun dibandingkan pembagian 80:20.
2. **Kelas 1 (Positif)**: Memiliki recall tinggi (1.0), menunjukkan semua contoh positif berhasil dikenali, tetapi presisi cukup rendah (0.67), menunjukkan adanya prediksi positif palsu.
3. **Kelas 3 (Netral)**: Presisi tinggi (1.0), tetapi recall rendah (0.33), yang berarti model sangat selektif dalam memilih contoh kelas ini.

#### Prediksi Aspek

1. **Akurasi**: 0.6190 (sekitar 62%) – performa sedikit meningkat dibandingkan pembagian 80:20.
2. **Kelas 1 (Akademik)**: Tidak ada prediksi benar untuk kelas ini (presisi dan recall sama-sama 0.0).
3. **Kelas 2 (Fasilitas)**: Memiliki presisi tinggi (0.80) dan recall sedang (0.50).
4. **Kelas 4 (Umum)**: Recall tinggi (0.82), tetapi presisi agak rendah (0.64), menunjukkan kecenderungan model mengklasifikasikan ke kelas "umum".

### Kesimpulan

1. **Performa SVM pada Sentimen dan Aspek**: Model SVM menunjukkan performa yang lebih baik pada prediksi sentimen dibandingkan aspek. Pada aspek, model cenderung mengklasifikasikan lebih sering ke kelas "umum," menunjukkan bahwa model mungkin mengalami kesulitan dalam menangani variasi aspek lain.
  
2. **Pengaruh Pembagian Data**: Pembagian data **70:30** memberikan hasil lebih seimbang antara presisi dan recall dibandingkan pembagian **80:20** dan **90:10**, di mana kelas minoritas (seperti "negatif" dan "akademik") sering terlewat.

3. **Rekomendasi**: Mengingat adanya bias ke kelas tertentu, penulis mungkin mempertimbangkan untuk menyeimbangkan kelas atau menggunakan representasi fitur yang lebih kuat seperti IndoBERT untuk meningkatkan performa dalam mengenali variasi aspek dan sentimen yang lebih kompleks.


## Langkah Eksperimen SVM dengan IndoBERT
Deskripsi Awal:
Penulis akan melakukan representasi teks menggunakan IndoBERT untuk mengekstrak fitur dari teks pada kolom cleaned_review. Setelah itu, fitur ini akan digunakan sebagai input untuk SVM dalam melakukan prediksi sentimen dan aspek.

### Penjelasan Script
1. Ekstraksi Fitur IndoBERT: Script mengekstrak fitur dari IndoBERT menggunakan mean dari hidden state terakhir, menghasilkan representasi numerik untuk setiap teks.
2. Pelatihan dan Pengujian SVM: Model SVM dilatih pada fitur IndoBERT dari data training, kemudian diuji pada data testing untuk menghasilkan akurasi dan laporan klasifikasi untuk prediksi sentimen dan aspek.
3. Evaluasi pada Pembagian Data Berbeda: Script menjalankan evaluasi pada pembagian data 70:30, 80:20, dan 90:10 untuk memberikan wawasan tentang kinerja model pada berbagai proporsi data.
Setelah menjalankan script ini, penulis dapat menganalisis hasil dari representasi IndoBERT dan melihat apakah representasi ini meningkatkan performa model dalam memprediksi sentimen dan aspek dibandingkan dengan metode TF-IDF.

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Load tokenizer dan model IndoBERT
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
indobert_model = AutoModel.from_pretrained("indobenchmark/indobert-base-p1")

# Fungsi untuk mengekstrak fitur IndoBERT dari teks
def extract_features(text_data):
    tokens = tokenizer(
        text_data.tolist(),
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )
    with torch.no_grad():
        outputs = indobert_model(**tokens)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# Fungsi untuk melatih dan menguji model SVM menggunakan fitur IndoBERT
def train_and_evaluate_svm_bert(train_data, test_data, target_column):
    # Ekstraksi fitur IndoBERT
    X_train = extract_features(train_data['cleaned_review'])
    y_train = train_data[target_column]
    X_test = extract_features(test_data['cleaned_review'])
    y_test = test_data[target_column]

    # Inisialisasi dan pelatihan model SVM
    model = SVC()
    model.fit(X_train, y_train)

    # Prediksi dan evaluasi
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    return accuracy, report

# Eksperimen SVM-IndoBERT untuk setiap kombinasi data dan target (sentimen & aspek)
for split, (train_data, test_data) in datasets.items():
    print(f"\nEvaluasi SVM-IndoBERT pada pembagian data {split} untuk prediksi Sentimen:")
    accuracy, report = train_and_evaluate_svm_bert(train_data, test_data, 'sentimen_encoded')
    print(f"Akurasi: {accuracy}\n{report}")

    print(f"\nEvaluasi SVM-IndoBERT pada pembagian data {split} untuk prediksi Aspek:")
    accuracy, report = train_and_evaluate_svm_bert(train_data, test_data, 'aspek_encoded')
    print(f"Akurasi: {accuracy}\n{report}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Evaluasi SVM-IndoBERT pada pembagian data 70:30 untuk prediksi Sentimen:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Akurasi: 0.7213114754098361
              precision    recall  f1-score   support

           1       0.75      0.91      0.82        44
           2       0.00      0.00      0.00         1
           3       0.50      0.25      0.33        16

    accuracy                           0.72        61
   macro avg       0.42      0.39      0.39        61
weighted avg       0.68      0.72      0.68        61


Evaluasi SVM-IndoBERT pada pembagian data 70:30 untuk prediksi Aspek:
Akurasi: 0.6229508196721312
              precision    recall  f1-score   support

           1       1.00      0.09      0.17        11
           2       0.54      0.59      0.57        22
           4       0.67      0.86      0.75        28

    accuracy                           0.62        61
   macro avg       0.74      0.51      0.49        61
weighted avg       0.68      0.62      0.58        61


Evaluasi SVM-IndoBERT pada pembagian data 80:20 untuk prediksi Sentimen:
Akurasi: 0.7317073170731707
         

## Interpretasi

### Evaluasi SVM-IndoBERT untuk tiap section Pembagian Data (70:30), (80:20), (90:10)

#### Prediksi Sentimen tiap pembagian data training:testing

1. **Pembagian Data 70:30**
   - **Akurasi**: 0.72 (72%) - Hasil akurasi menunjukkan model cukup baik dalam memprediksi sentimen dengan pembagian data ini.
   - **Kelas 1 (Positif)**: Presisi tinggi (0.75) dan recall tinggi (0.91), menunjukkan bahwa model dapat mengenali dan memprediksi kelas "positif" secara konsisten.
   - **Kelas 2 (Negatif)**: Dengan satu contoh, model gagal memprediksi kelas ini, memberikan presisi dan recall 0.0.
   - **Kelas 3 (Netral)**: Presisi (0.50) dan recall (0.25) mengindikasikan model mengalami kesulitan dalam mengenali kelas netral.
   - **Macro dan Weighted Average**: Macro avg menunjukkan rata-rata performa pada setiap kelas secara independen, sedangkan weighted avg menunjukkan kinerja yang lebih tinggi, menunjukkan model lebih berfokus pada kelas mayoritas.

2. **Pembagian Data 80:20**
   - **Akurasi**: 0.73 (73%) - Sedikit meningkat dari pembagian 70:30, menandakan pembagian data ini tetap efektif.
   - **Kelas 1 (Positif)**: Presisi (0.72) dan recall tinggi (1.00) menunjukkan model mampu mengenali seluruh contoh positif dengan baik.
   - **Kelas 3 (Netral)**: Presisi tinggi (1.00) tetapi recall sangat rendah (0.15), menunjukkan adanya bias terhadap kelas "positif".
   - **Macro dan Weighted Average**: Macro avg menurun di recall dan F1-score karena adanya bias ke kelas "positif", sementara weighted avg lebih tinggi karena jumlah contoh yang lebih besar pada kelas mayoritas.

3. **Pembagian Data 90:10**
   - **Akurasi**: 0.62 (62%) - Menunjukkan performa model menurun pada pembagian data ini.
   - **Kelas 1 (Positif)**: Recall sempurna (1.00), namun presisi sedang (0.60), menunjukkan model menghasilkan prediksi positif palsu untuk kelas ini.
   - **Kelas 3 (Netral)**: Presisi tinggi (1.00), tetapi recall sangat rendah (0.11), menunjukkan bahwa model kesulitan mengenali seluruh contoh netral dengan benar.
   - **Macro dan Weighted Average**: Macro avg menunjukkan performa keseluruhan yang rendah pada kelas minoritas, sementara weighted avg lebih baik karena adanya bias ke kelas mayoritas.

#### Prediksi Aspek tiap pembagian data training:testing

1. **Pembagian Data 70:30**
   - **Akurasi**: 0.62 (62%) - Hasil ini menunjukkan performa yang relatif rendah untuk aspek dengan pembagian data ini.
   - **Kelas 1 (Akademik)**: Tingkat presisi tinggi (1.00) tetapi recall sangat rendah (0.09), menunjukkan bahwa model gagal mendeteksi sebagian besar contoh kelas "akademik".
   - **Kelas 2 (Fasilitas)**: Presisi (0.54) dan recall sedang (0.59), menunjukkan model memiliki hasil moderat untuk aspek fasilitas.
   - **Kelas 4 (Umum)**: Presisi (0.67) dan recall tinggi (0.86), menunjukkan model sering mengklasifikasikan data ke dalam kelas ini.
   - **Macro dan Weighted Average**: Macro avg rendah pada recall dan F1-score, menunjukkan model gagal dalam mengenali beberapa kelas dengan presisi dan recall yang seimbang.

2. **Pembagian Data 80:20**
   - **Akurasi**: 0.61 (61%) - Performa serupa dengan pembagian 70:30, menunjukkan kesulitan pada aspek.
   - **Kelas 1 (Akademik)**: Presisi cukup (0.67) tetapi recall rendah (0.25), menunjukkan bahwa sebagian besar contoh kelas ini tidak dikenali.
   - **Kelas 2 (Fasilitas)**: Presisi dan recall cukup seimbang (sekitar 0.55).
   - **Kelas 4 (Umum)**: Recall tinggi (0.83), menunjukkan model sering memilih kelas "umum" untuk data uji.
   - **Macro dan Weighted Average**: Hasil weighted avg pada akurasi (0.61) menunjukkan ketidakseimbangan model dalam mengenali semua kelas.

3. **Pembagian Data 90:10**
   - **Akurasi**: 0.62 (62%) - Mirip dengan pembagian data lainnya, performa tidak banyak meningkat.
   - **Kelas 1 (Akademik)**: Tidak ada prediksi yang benar untuk kelas ini, mengakibatkan presisi dan recall sama-sama 0.0.
   - **Kelas 2 (Fasilitas)**: Presisi (0.57) dan recall (0.50) menunjukkan hasil moderat.
   - **Kelas 4 (Umum)**: Recall tinggi (0.82) dengan presisi moderat (0.69), menunjukkan model cenderung mengklasifikasikan data ke kelas "umum".
   - **Macro dan Weighted Average**: Macro avg rendah di semua metrik, mengindikasikan ketidakseimbangan prediksi pada kelas minoritas.

---

### Kesimpulan
Secara keseluruhan, model **SVM dengan IndoBERT** menunjukkan performa yang lebih kuat dalam prediksi sentimen pada kelas mayoritas (misalnya, kelas "positif"), tetapi memiliki kesulitan untuk mengenali kelas minoritas dalam `sentimen` dan `aspek`. Selain itu, model juga cenderung mengklasifikasikan data ke kelas "umum" pada prediksi aspek. Hasil ini menunjukkan perlunya pendekatan tambahan untuk menangani ketidakseimbangan kelas.


| Pembagian Data | Prediksi | SVM TF-IDF Akurasi | SVM IndoBERT Akurasi |
|----------------|----------|---------------------|-----------------------|
| 70:30          | Sentimen | 0.74               | 0.72                 |
| 70:30          | Aspek    | 0.61               | 0.62                 |
| 80:20          | Sentimen | 0.76               | 0.73                 |
| 80:20          | Aspek    | 0.59               | 0.61                 |
| 90:10          | Sentimen | 0.71               | 0.62                 |
| 90:10          | Aspek    | 0.62               | 0.62                 |

### Interpretasi

- Pada prediksi **sentimen**, representasi **TF-IDF** umumnya memberikan akurasi yang sedikit lebih tinggi daripada **IndoBERT** di sebagian besar pembagian data.
- Pada prediksi **aspek**, **IndoBERT** memiliki performa yang lebih baik pada pembagian data 70:30 dan 80:20, meskipun perbedaannya relatif kecil.

Penggunaan **IndoBERT** meningkatkan akurasi dalam beberapa kasus, terutama pada prediksi **aspek**, namun hasilnya masih cenderung setara dengan TF-IDF pada prediksi **sentimen**.


# Modelling dengan SVM-PSO

Untuk melanjutkan dengan eksperimen SVM-PSO menggunakan TF-IDF, penulis akan menggunakan Particle Swarm Optimization (PSO) untuk mengoptimalkan parameter utama SVM, yaitu C dan gamma. Berikut adalah langkah-langkah untuk mengatur SVM dengan optimasi PSO menggunakan TF-IDF sebagai representasi fitur.

## Script Modelling SVM-PSO dengan TF-IDF

In [ ]:
import optuna
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Inisialisasi TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Fungsi untuk mengoptimalkan SVM dengan PSO menggunakan Optuna
def svm_with_pso_tfidf(train_data, test_data, target_column):
    # Transformasi teks ke TF-IDF
    X_train = tfidf_vectorizer.fit_transform(train_data['cleaned_review'])
    y_train = train_data[target_column]
    X_test = tfidf_vectorizer.transform(test_data['cleaned_review'])
    y_test = test_data[target_column]

    # Fungsi optimasi untuk PSO
    def objective(trial):
        C = trial.suggest_loguniform('C', 1e-5, 1e2)
        gamma = trial.suggest_loguniform('gamma', 1e-5, 1e1)
        model = SVC(C=C, gamma=gamma)
        model.fit(X_train, y_train)
        return model.score(X_test, y_test)

    # Melakukan optimasi menggunakan Optuna
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=10)

    # Model dengan parameter terbaik
    best_params = study.best_params
    model = SVC(**best_params)
    model.fit(X_train, y_train)

    # Prediksi dan evaluasi
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    return accuracy, report, best_params

# Melakukan eksperimen SVM-PSO TF-IDF untuk setiap kombinasi data dan target (sentimen & aspek)
for split, (train_data, test_data) in datasets.items():
    print(f"\nEvaluasi SVM-PSO TF-IDF pada pembagian data {split} untuk prediksi Sentimen:")
    accuracy, report, params = svm_with_pso_tfidf(train_data, test_data, 'sentimen_encoded')
    print(f"Akurasi: {accuracy:.2f}\n{report}\nParameter Terbaik: {params}")

    print(f"\nEvaluasi SVM-PSO TF-IDF pada pembagian data {split} untuk prediksi Aspek:")
    accuracy, report, params = svm_with_pso_tfidf(train_data, test_data, 'aspek_encoded')
    print(f"Akurasi: {accuracy:.2f}\n{report}\nParameter Terbaik: {params}")


[I 2024-10-30 07:08:45,326] A new study created in memory with name: no-name-a4a77dc6-ade8-4f94-9096-0c0a5583f492
<ipython-input-132-18c439b6b489>:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e2)
<ipython-input-132-18c439b6b489>:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform('gamma', 1e-5, 1e1)
[I 2024-10-30 07:08:45,341] Trial 0 finished with value: 0.7213114754098361 and parameters: {'C': 0.0037883790817543417, 'gamma': 0.10816323714187714}. Best is trial 0 with value: 0.7213114754098361.
<ipython-input-132-18c439b6b489>:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0


Evaluasi SVM-PSO TF-IDF pada pembagian data 70:30 untuk prediksi Sentimen:
Akurasi: 0.77
              precision    recall  f1-score   support

           1       0.76      1.00      0.86        44
           2       0.00      0.00      0.00         1
           3       1.00      0.19      0.32        16

    accuracy                           0.77        61
   macro avg       0.59      0.40      0.39        61
weighted avg       0.81      0.77      0.71        61

Parameter Terbaik: {'C': 1.3954845580411734, 'gamma': 0.1335100234463851}

Evaluasi SVM-PSO TF-IDF pada pembagian data 70:30 untuk prediksi Aspek:


<ipython-input-132-18c439b6b489>:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e2)
<ipython-input-132-18c439b6b489>:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform('gamma', 1e-5, 1e1)
[I 2024-10-30 07:08:45,534] Trial 2 finished with value: 0.6065573770491803 and parameters: {'C': 31.50415763506532, 'gamma': 0.9728679219071423}. Best is trial 2 with value: 0.6065573770491803.
<ipython-input-132-18c439b6b489>:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(.

Akurasi: 0.61
              precision    recall  f1-score   support

           1       0.50      0.36      0.42        11
           2       0.75      0.41      0.53        22
           4       0.59      0.86      0.70        28

    accuracy                           0.61        61
   macro avg       0.61      0.54      0.55        61
weighted avg       0.63      0.61      0.59        61

Parameter Terbaik: {'C': 31.50415763506532, 'gamma': 0.9728679219071423}

Evaluasi SVM-PSO TF-IDF pada pembagian data 80:20 untuk prediksi Sentimen:
Akurasi: 0.78
              precision    recall  f1-score   support

           1       0.85      0.82      0.84        28
           3       0.64      0.69      0.67        13

    accuracy                           0.78        41
   macro avg       0.75      0.76      0.75        41
weighted avg       0.79      0.78      0.78        41

Parameter Terbaik: {'C': 14.132428807733456, 'gamma': 0.26245799624738486}

Evaluasi SVM-PSO TF-IDF pada pembagian 

<ipython-input-132-18c439b6b489>:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e2)
<ipython-input-132-18c439b6b489>:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform('gamma', 1e-5, 1e1)
[I 2024-10-30 07:08:45,870] Trial 1 finished with value: 0.43902439024390244 and parameters: {'C': 0.618898162440413, 'gamma': 1.0408383942226772e-05}. Best is trial 0 with value: 0.43902439024390244.
<ipython-input-132-18c439b6b489>:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_f

Akurasi: 0.54
              precision    recall  f1-score   support

           1       0.33      0.25      0.29         8
           2       0.64      0.47      0.54        15
           4       0.54      0.72      0.62        18

    accuracy                           0.54        41
   macro avg       0.50      0.48      0.48        41
weighted avg       0.54      0.54      0.52        41

Parameter Terbaik: {'C': 7.2116438458097765, 'gamma': 0.9936403375059631}

Evaluasi SVM-PSO TF-IDF pada pembagian data 90:10 untuk prediksi Sentimen:


<ipython-input-132-18c439b6b489>:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e2)
<ipython-input-132-18c439b6b489>:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform('gamma', 1e-5, 1e1)
[I 2024-10-30 07:08:46,319] Trial 6 finished with value: 0.5714285714285714 and parameters: {'C': 0.0015245450011406806, 'gamma': 0.00451685755253732}. Best is trial 4 with value: 0.7142857142857143.
<ipython-input-132-18c439b6b489>:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_fl

Akurasi: 0.71
              precision    recall  f1-score   support

           1       0.67      1.00      0.80        12
           3       1.00      0.33      0.50         9

    accuracy                           0.71        21
   macro avg       0.83      0.67      0.65        21
weighted avg       0.81      0.71      0.67        21

Parameter Terbaik: {'C': 1.1845313385137783, 'gamma': 3.2997626468387447}

Evaluasi SVM-PSO TF-IDF pada pembagian data 90:10 untuk prediksi Aspek:
Akurasi: 0.62
              precision    recall  f1-score   support

           1       0.25      0.50      0.33         2
           2       0.67      0.50      0.57         8
           4       0.73      0.73      0.73        11

    accuracy                           0.62        21
   macro avg       0.55      0.58      0.54        21
weighted avg       0.66      0.62      0.63        21

Parameter Terbaik: {'C': 7.107816755062824, 'gamma': 0.19006209836803753}


### Interpretasi

Interpretasi hasil evaluasi di atas mencakup analisis akurasi serta metrik evaluasi lainnya (presisi, recall, dan F1-score) untuk model SVM-PSO dalam memprediksi sentimen dan aspek pada setiap kombinasi pembagian data (70:30, 80:20, dan 90:10) dengan representasi TF-IDF. Berikut penjelasan per bagian:

### Evaluasi SVM-PSO TF-IDF pada Pembagian Data 70:30
#### Prediksi Sentimen

1. **Akurasi**: 0.77 (sekitar 77%) – Model berhasil memprediksi sentimen dengan benar pada sekitar 77% data uji.
2. **Kelas 1 (Positif)**: Presisi 0.76 dan recall 1.00, menunjukkan bahwa model sangat baik dalam mendeteksi kelas positif, dengan minim kesalahan prediksi positif palsu.
3. **Kelas 2 (Negatif)**: Presisi dan recall keduanya 0.00 karena hanya ada satu contoh kelas ini, dan model gagal memprediksinya.
4. **Kelas 3 (Netral)**: Presisi 1.00, tetapi recall rendah di 0.19, menunjukkan model mengalami kesulitan mengidentifikasi sebagian besar contoh kelas netral.

#### Prediksi Aspek

1. **Akurasi**: 0.61 (sekitar 61%) – Model mampu memprediksi aspek dengan benar untuk sekitar 61% data uji.
2. **Kelas 1 (Akademik)**: Presisi 0.50 dan recall 0.36, menunjukkan model kesulitan membedakan kelas ini dari kelas lainnya.
3. **Kelas 2 (Fasilitas)**: Presisi cukup baik di 0.75, tetapi recall hanya 0.41, menunjukkan ada beberapa kesalahan pengenalan pada kelas fasilitas.
4. **Kelas 4 (Umum)**: Recall tinggi (0.86) menunjukkan kecenderungan model untuk mengklasifikasikan ke kelas ini, namun presisi yang lebih rendah di 0.59.

### Evaluasi SVM-PSO TF-IDF pada Pembagian Data 80:20
#### Prediksi Sentimen

1. **Akurasi**: 0.78 (sekitar 78%) – Tingkat akurasi sedikit meningkat dibandingkan pembagian 70:30.
2. **Kelas 1 (Positif)**: Presisi dan recall tinggi (0.85 dan 0.82), menunjukkan bahwa model mengenali kelas ini dengan baik.
3. **Kelas 3 (Netral)**: Presisi 0.64 dan recall 0.69, menunjukkan performa yang lebih baik dalam mendeteksi kelas netral dibandingkan pembagian data sebelumnya.

#### Prediksi Aspek

1. **Akurasi**: 0.54 (sekitar 54%) – Performa model sedikit menurun dibandingkan pembagian 70:30.
2. **Kelas 1 (Akademik)**: Presisi rendah (0.33) dan recall 0.25, menunjukkan banyak contoh kelas akademik yang sulit dikenali oleh model.
3. **Kelas 2 (Fasilitas)**: Presisi 0.64 dan recall 0.47, menunjukkan adanya bias pengenalan ke kelas ini.
4. **Kelas 4 (Umum)**: Recall 0.72 menunjukkan bahwa kelas ini sering dikenali dengan baik, meskipun presisi 0.54 cukup rendah.

### Evaluasi SVM-PSO TF-IDF pada Pembagian Data 90:10
#### Prediksi Sentimen

1. **Akurasi**: 0.71 (sekitar 71%) – Performa menurun sedikit dibandingkan pembagian 80:20.
2. **Kelas 1 (Positif)**: Recall tinggi (1.00) menunjukkan model berhasil mengidentifikasi semua contoh positif, tetapi presisi sedikit lebih rendah di 0.67.
3. **Kelas 3 (Netral)**: Presisi tinggi (1.00) tetapi recall 0.33, menunjukkan model sangat selektif dalam memilih contoh kelas ini.

#### Prediksi Aspek

1. **Akurasi**: 0.62 (sekitar 62%) – Akurasi sedikit meningkat dibandingkan pembagian 80:20.
2. **Kelas 1 (Akademik)**: Presisi 0.25 dan recall 0.50, menunjukkan beberapa prediksi salah di kelas ini.
3. **Kelas 2 (Fasilitas)**: Presisi 0.67 dan recall 0.50, menunjukkan ketidakseimbangan dalam pengenalan kelas ini.
4. **Kelas 4 (Umum)**: Recall cukup tinggi di 0.73, menunjukkan kecenderungan model mengklasifikasikan ke kelas umum.

### Kesimpulan
1. **Performa SVM-PSO pada Sentimen dan Aspek**: Model SVM-PSO menunjukkan performa yang cukup baik pada prediksi sentimen dengan tingkat akurasi tertinggi 78%. Pada aspek, model cenderung lebih sering mengklasifikasikan ke kelas "umum," yang dapat menunjukkan kesulitan dalam membedakan variasi aspek lainnya.
2. **Pengaruh Pembagian Data**: Pembagian data 70:30 memberikan hasil paling seimbang di antara presisi dan recall dibandingkan pembagian data lainnya. Pembagian 90:10 menunjukkan performa yang lebih rendah dalam mendeteksi kelas-kelas minoritas, terutama pada prediksi aspek.

**Rekomendasi**: Mengingat adanya bias ke kelas tertentu dan kendala dalam pengenalan beberapa aspek, penulis mungkin dapat mempertimbangkan untuk menyeimbangkan kelas atau menggunakan representasi fitur lain seperti IndoBERT untuk meningkatkan performa dalam mengenali variasi aspek dan sentimen yang lebih kompleks.


## Script Modelling SVM-PSO dengan IndoBERT

Untuk meningkatkan performa prediksi sentimen dan aspek, model Support Vector Machine (SVM) akan dioptimalkan menggunakan Particle Swarm Optimization (PSO) bersama dengan representasi fitur berbasis IndoBERT. Kombinasi SVM dengan PSO memungkinkan kita untuk mencari parameter optimal dari model SVM, seperti nilai hyperparameter C dan gamma, yang dapat meningkatkan akurasi model. IndoBERT, di sisi lain, merupakan model bahasa yang sudah dilatih khusus untuk Bahasa Indonesia dan akan digunakan untuk melakukan transformasi teks menjadi vektor fitur yang lebih representatif.

Berikut adalah script yang akan melakukan proses ini. Pertama, kita akan mengimpor library yang diperlukan, lalu mempersiapkan pipeline pemodelan yang mencakup IndoBERT untuk ekstraksi fitur, PSO untuk optimasi hyperparameter, dan SVM sebagai model klasifikasi.



In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, BertModel
import torch

# Memuat model IndoBERT dan tokenizer
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
model = BertModel.from_pretrained("indobenchmark/indobert-base-p1")

# Tokenisasi kolom 'cleaned_review'
tokens_labeled = tokenizer(
    data_labeled['cleaned_review'].tolist(),
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

# Menghasilkan embedding
with torch.no_grad():
    embeddings = model(**tokens_labeled).last_hidden_state.mean(dim=1).numpy()

# Embedding sekarang berisi representasi untuk setiap review dalam kolom data_labeled['cleaned_review']


In [ ]:
# Import library yang diperlukan
import optuna
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from transformers import BertTokenizer, BertModel
import torch
from sklearn.model_selection import train_test_split
from optuna.samplers import TPESampler

# Muat data
# Data `reviews` dan `labels_sentiment` dan `labels_aspect` harus sudah disiapkan, dengan `reviews` sebagai teks input
# dan `labels_sentiment` dan `labels_aspect` masing-masing berisi label untuk sentimen dan aspek.
# Pastikan IndoBERT sudah terinstal dengan: `pip install transformers`

# Load data yang sudah disiapkan
reviews = data_labeled['cleaned_review'].tolist()
labels_sentiment = data_labeled['sentimen'].tolist()
labels_aspect = data_labeled['aspek'].tolist()

# Inisialisasi IndoBERT tokenizer dan model
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
model = BertModel.from_pretrained("indobenchmark/indobert-base-p1")

# Fungsi untuk mendapatkan representasi vektor dari teks menggunakan IndoBERT
def get_indo_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()  # Ambil [CLS] token sebagai representasi teks


# Mengubah data teks menjadi embedding
X = [get_indo_bert_embeddings(text) for text in reviews]
y_sentiment = labels_sentiment
y_aspect = labels_aspect

# Split data untuk training dan testing
X_train, X_test, y_train_sent, y_test_sent = train_test_split(X, y_sentiment, test_size=0.3, random_state=42)
X_train_aspect, X_test_aspect, y_train_aspect, y_test_aspect = train_test_split(X, y_aspect, test_size=0.3, random_state=42)

# Fungsi untuk melakukan optimasi hyperparameter SVM dengan PSO
def objective(trial, X_train, y_train, X_test, y_test):
    C = trial.suggest_float('C', 1e-5, 1e2, log=True)
    gamma = trial.suggest_float('gamma', 1e-5, 1e1, log=True)

    model = SVC(C=C, gamma=gamma, kernel='rbf')
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Optimasi untuk prediksi sentimen
study_sentiment = optuna.create_study(direction='maximize', sampler=TPESampler())
study_sentiment.optimize(lambda trial: objective(trial, X_train, y_train_sent, X_test, y_test_sent), n_trials=10)

# Hasil terbaik untuk prediksi sentimen
best_params_sentiment = study_sentiment.best_params
print("Parameter Terbaik Sentimen:", best_params_sentiment)

# Evaluasi model terbaik untuk sentimen
model_sentiment = SVC(**best_params_sentiment, kernel='rbf')
model_sentiment.fit(X_train, y_train_sent)
y_pred_sentiment = model_sentiment.predict(X_test)
print("Akurasi Sentimen:", accuracy_score(y_test_sent, y_pred_sentiment))
print(classification_report(y_test_sent, y_pred_sentiment))

# Optimasi untuk prediksi aspek
study_aspect = optuna.create_study(direction='maximize', sampler=TPESampler())
study_aspect.optimize(lambda trial: objective(trial, X_train_aspect, y_train_aspect, X_test_aspect, y_test_aspect), n_trials=10)

# Hasil terbaik untuk prediksi aspek
best_params_aspect = study_aspect.best_params
print("Parameter Terbaik Aspek:", best_params_aspect)

# Evaluasi model terbaik untuk aspek
model_aspect = SVC(**best_params_aspect, kernel='rbf')
model_aspect.fit(X_train_aspect, y_train_aspect)
y_pred_aspect = model_aspect.predict(X_test_aspect)
print("Akurasi Aspek:", accuracy_score(y_test_aspect, y_pred_aspect))
print(classification_report(y_test_aspect, y_pred_aspect))


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
[I 2024-10-30 08:09:19,250] A new study created in memory with name: no-name-65f1654c-925a-43ca-941c-096d66bf8fe7
[I 2024-10-30 08:09:19,269] Trial 0 finished with value: 0.7377049180327869 and parameters: {'C': 18.092213862706057, 'gamma': 1.4026534017642651}. Best is trial 0 with value: 0.7377049180327869.
[I 2024-10-30 08:09:19,289] Trial 1 finished with value: 0.7377049180327869 and parameters: {'C': 30.178600091364643, 'gamma': 0.8779888334344593}. Best is trial 0 with value: 0.7377049180327869.
[I 2024-10-30 08:09:19,309] Trial 2 finished with value: 0.7213114754098361 and parameters: {'C': 0.0002

Parameter Terbaik Sentimen: {'C': 18.092213862706057, 'gamma': 1.4026534017642651}
Akurasi Sentimen: 0.7377049180327869
              precision    recall  f1-score   support

     negatif       0.00      0.00      0.00         1
      netral       1.00      0.06      0.12        16
     positif       0.73      1.00      0.85        44

    accuracy                           0.74        61
   macro avg       0.58      0.35      0.32        61
weighted avg       0.79      0.74      0.64        61

Parameter Terbaik Aspek: {'C': 10.808930323949014, 'gamma': 7.991062309877341e-05}
Akurasi Aspek: 0.6229508196721312
              precision    recall  f1-score   support

    akademik       0.00      0.00      0.00        11
   fasilitas       0.57      0.73      0.64        22
        umum       0.69      0.79      0.73        28

    accuracy                           0.62        61
   macro avg       0.42      0.50      0.46        61
weighted avg       0.52      0.62      0.57        61



In [ ]:
# Inisialisasi model dengan class_weight untuk menangani ketidakseimbangan data
model_sentiment = SVC(**best_params_sentiment, kernel='rbf', class_weight='balanced')
model_sentiment.fit(X_train, y_train_sent)
y_pred_sentiment = model_sentiment.predict(X_test)
print("Akurasi Sentimen:", accuracy_score(y_test_sent, y_pred_sentiment))
print(classification_report(y_test_sent, y_pred_sentiment))


Akurasi Sentimen: 0.7213114754098361
              precision    recall  f1-score   support

     negatif       0.00      0.00      0.00         1
      netral       0.50      0.06      0.11        16
     positif       0.73      0.98      0.83        44

    accuracy                           0.72        61
   macro avg       0.41      0.35      0.32        61
weighted avg       0.66      0.72      0.63        61



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import optuna
from optuna.samplers import TPESampler

# Fungsi untuk optimasi hyperparameter SVM dengan PSO
def objective(trial, X_train, y_train, X_test, y_test):
    C = trial.suggest_float('C', 1e-5, 1e2, log=True)
    gamma = trial.suggest_float('gamma', 1e-5, 1e1, log=True)

    model = SVC(C=C, gamma=gamma, kernel='rbf', class_weight='balanced')
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Fungsi untuk melatih dan evaluasi model dengan berbagai rasio split
def train_and_evaluate(X, y_sentiment, y_aspect, test_size, label):
    # Split data
    X_train, X_test, y_train_sent, y_test_sent = train_test_split(X, y_sentiment, test_size=test_size, random_state=42)
    X_train_aspect, X_test_aspect, y_train_aspect, y_test_aspect = train_test_split(X, y_aspect, test_size=test_size, random_state=42)

    print(f"Evaluasi dengan split {int((1-test_size)*10)}:{int(test_size*10)} untuk {label}:")

    # Optimasi untuk prediksi sentimen
    study_sentiment = optuna.create_study(direction='maximize', sampler=TPESampler())
    study_sentiment.optimize(lambda trial: objective(trial, X_train, y_train_sent, X_test, y_test_sent), n_trials=10)
    best_params_sentiment = study_sentiment.best_params
    print("Parameter Terbaik Sentimen:", best_params_sentiment)

    # Evaluasi model terbaik untuk sentimen
    model_sentiment = SVC(**best_params_sentiment, kernel='rbf', class_weight='balanced')
    model_sentiment.fit(X_train, y_train_sent)
    y_pred_sentiment = model_sentiment.predict(X_test)
    print("Akurasi Sentimen:", accuracy_score(y_test_sent, y_pred_sentiment))
    print(classification_report(y_test_sent, y_pred_sentiment))

    # Optimasi untuk prediksi aspek
    study_aspect = optuna.create_study(direction='maximize', sampler=TPESampler())
    study_aspect.optimize(lambda trial: objective(trial, X_train_aspect, y_train_aspect, X_test_aspect, y_test_aspect), n_trials=10)
    best_params_aspect = study_aspect.best_params
    print("Parameter Terbaik Aspek:", best_params_aspect)

    # Evaluasi model terbaik untuk aspek
    model_aspect = SVC(**best_params_aspect, kernel='rbf', class_weight='balanced')
    model_aspect.fit(X_train_aspect, y_train_aspect)
    y_pred_aspect = model_aspect.predict(X_test_aspect)
    print("Akurasi Aspek:", accuracy_score(y_test_aspect, y_pred_aspect))
    print(classification_report(y_test_aspect, y_pred_aspect))

# Menjalankan pelatihan dan evaluasi untuk split 7:3, 8:2, dan 9:1
print("Pelatihan dan evaluasi SVM-PSO dengan IndoBERT:\n")
train_and_evaluate(X, y_sentiment, y_aspect, test_size=0.3, label="7:3")
train_and_evaluate(X, y_sentiment, y_aspect, test_size=0.2, label="8:2")
train_and_evaluate(X, y_sentiment, y_aspect, test_size=0.1, label="9:1")


[I 2024-10-30 08:11:47,089] A new study created in memory with name: no-name-4a5d9af7-5074-479d-8b6e-ad51cd18f693
[I 2024-10-30 08:11:47,169] Trial 0 finished with value: 0.7213114754098361 and parameters: {'C': 20.567545840992864, 'gamma': 1.0924608271237668}. Best is trial 0 with value: 0.7213114754098361.
[I 2024-10-30 08:11:47,226] Trial 1 finished with value: 0.639344262295082 and parameters: {'C': 36.79395695262706, 'gamma': 2.5432963501873615e-05}. Best is trial 0 with value: 0.7213114754098361.
[I 2024-10-30 08:11:47,271] Trial 2 finished with value: 0.7213114754098361 and parameters: {'C': 0.3769818855017601, 'gamma': 3.055378652770113}. Best is trial 0 with value: 0.7213114754098361.


Pelatihan dan evaluasi SVM-PSO dengan IndoBERT:

Evaluasi dengan split 7:3 untuk 7:3:


[I 2024-10-30 08:11:47,328] Trial 3 finished with value: 0.7213114754098361 and parameters: {'C': 22.932953246772072, 'gamma': 0.01798341702833216}. Best is trial 0 with value: 0.7213114754098361.
[I 2024-10-30 08:11:47,366] Trial 4 finished with value: 0.26229508196721313 and parameters: {'C': 0.0007182282044090647, 'gamma': 0.014423681529515592}. Best is trial 0 with value: 0.7213114754098361.
[I 2024-10-30 08:11:47,393] Trial 5 finished with value: 0.16393442622950818 and parameters: {'C': 2.9867451133921212e-05, 'gamma': 0.011057202079909258}. Best is trial 0 with value: 0.7213114754098361.
[I 2024-10-30 08:11:47,430] Trial 6 finished with value: 0.26229508196721313 and parameters: {'C': 0.001466607548855752, 'gamma': 0.0018633303989951744}. Best is trial 0 with value: 0.7213114754098361.
[I 2024-10-30 08:11:47,465] Trial 7 finished with value: 0.7049180327868853 and parameters: {'C': 10.162338976515832, 'gamma': 0.0016566374154422042}. Best is trial 0 with value: 0.721311475409836

Parameter Terbaik Sentimen: {'C': 20.567545840992864, 'gamma': 1.0924608271237668}
Akurasi Sentimen: 0.7213114754098361
              precision    recall  f1-score   support

     negatif       0.00      0.00      0.00         1
      netral       0.50      0.06      0.11        16
     positif       0.73      0.98      0.83        44

    accuracy                           0.72        61
   macro avg       0.41      0.35      0.32        61
weighted avg       0.66      0.72      0.63        61



[I 2024-10-30 08:11:47,769] Trial 4 finished with value: 0.45901639344262296 and parameters: {'C': 4.166838839708268, 'gamma': 0.04764282646543922}. Best is trial 3 with value: 0.5573770491803278.
[I 2024-10-30 08:11:47,811] Trial 5 finished with value: 0.29508196721311475 and parameters: {'C': 2.336789233918595e-05, 'gamma': 0.000506277739235134}. Best is trial 3 with value: 0.5573770491803278.
[I 2024-10-30 08:11:47,860] Trial 6 finished with value: 0.5737704918032787 and parameters: {'C': 41.53220084137213, 'gamma': 0.008369162146560869}. Best is trial 6 with value: 0.5737704918032787.
[I 2024-10-30 08:11:47,913] Trial 7 finished with value: 0.45901639344262296 and parameters: {'C': 12.309744060685475, 'gamma': 1.716311907459989}. Best is trial 6 with value: 0.5737704918032787.
[I 2024-10-30 08:11:47,966] Trial 8 finished with value: 0.4918032786885246 and parameters: {'C': 0.1868660776646812, 'gamma': 0.006679285104410804}. Best is trial 6 with value: 0.5737704918032787.
[I 2024-10

...

Parameter Terbaik Aspek: {'C': 56.32338214971811, 'gamma': 0.20409321612050185}
Akurasi Aspek: 0.43902439024390244
              precision    recall  f1-score   support

    akademik       0.00      0.00      0.00         8
   fasilitas       0.00      0.00      0.00        15
        umum       0.44      1.00      0.61        18

    accuracy                           0.44        41
   macro avg       0.15      0.33      0.20        41
weighted avg       0.19      0.44      0.27        41

Evaluasi dengan split 9:1 untuk 9:1:


[I 2024-10-30 08:11:49,327] Trial 4 finished with value: 0.0 and parameters: {'C': 1.8437266527910226e-05, 'gamma': 0.042900833535661854}. Best is trial 0 with value: 0.5714285714285714.
[I 2024-10-30 08:11:49,381] Trial 5 finished with value: 0.0 and parameters: {'C': 0.0005542483848038593, 'gamma': 0.0012143683988359347}. Best is trial 0 with value: 0.5714285714285714.
[I 2024-10-30 08:11:49,436] Trial 6 finished with value: 0.0 and parameters: {'C': 0.00014562114497861366, 'gamma': 0.11442986745706184}. Best is trial 0 with value: 0.5714285714285714.
[I 2024-10-30 08:11:49,495] Trial 7 finished with value: 0.5714285714285714 and parameters: {'C': 2.0378308424504927e-05, 'gamma': 1.8766492299449056}. Best is trial 0 with value: 0.5714285714285714.
[I 2024-10-30 08:11:49,538] Trial 8 finished with value: 0.5714285714285714 and parameters: {'C': 0.004176435332994566, 'gamma': 1.6853637696272374e-05}. Best is trial 0 with value: 0.5714285714285714.
[I 2024-10-30 08:11:49,568] Trial 9 fi

Parameter Terbaik Sentimen: {'C': 15.85822289826863, 'gamma': 0.07762457957949022}
Akurasi Sentimen: 0.6190476190476191
              precision    recall  f1-score   support

      netral       1.00      0.11      0.20         9
     positif       0.60      1.00      0.75        12

    accuracy                           0.62        21
   macro avg       0.80      0.56      0.47        21
weighted avg       0.77      0.62      0.51        21



[I 2024-10-30 08:11:49,829] Trial 3 finished with value: 0.5238095238095238 and parameters: {'C': 6.017621134484686, 'gamma': 0.269200958959941}. Best is trial 1 with value: 0.5238095238095238.
[I 2024-10-30 08:11:49,881] Trial 4 finished with value: 0.09523809523809523 and parameters: {'C': 0.00017607207800057214, 'gamma': 0.0002946707237427205}. Best is trial 1 with value: 0.5238095238095238.
[I 2024-10-30 08:11:49,916] Trial 5 finished with value: 0.42857142857142855 and parameters: {'C': 0.1981812918666088, 'gamma': 0.0014572081806447062}. Best is trial 1 with value: 0.5238095238095238.
[I 2024-10-30 08:11:49,991] Trial 6 finished with value: 0.09523809523809523 and parameters: {'C': 0.000400083113439996, 'gamma': 0.10696078212337924}. Best is trial 1 with value: 0.5238095238095238.
[I 2024-10-30 08:11:50,063] Trial 7 finished with value: 0.09523809523809523 and parameters: {'C': 1.492329247993501e-05, 'gamma': 0.3152014754416859}. Best is trial 1 with value: 0.5238095238095238.
[I

Parameter Terbaik Aspek: {'C': 0.28684791029497186, 'gamma': 0.0019238226621660318}
Akurasi Aspek: 0.5714285714285714
              precision    recall  f1-score   support

    akademik       0.00      0.00      0.00         2
   fasilitas       0.56      0.62      0.59         8
        umum       0.78      0.64      0.70        11

    accuracy                           0.57        21
   macro avg       0.44      0.42      0.43        21
weighted avg       0.62      0.57      0.59        21



### Interpretasi

Interpretasi hasil evaluasi di atas mencakup analisis akurasi serta metrik evaluasi lainnya (presisi, recall, dan F1-score) untuk model SVM-PSO dalam memprediksi sentimen dan aspek pada setiap kombinasi pembagian data (70:30, 80:20, dan 90:10) dengan representasi IndoBERT. Berikut penjelasan per bagian:

### Evaluasi SVM-PSO IndoBERT pada Pembagian Data 70:30
#### Prediksi Sentimen

1. **Akurasi**: 0.7213 (sekitar 72.13%) – Model berhasil memprediksi sentimen dengan benar pada sekitar 72.13% data uji.
2. **Kelas Positif**: Presisi 0.73 dan recall 0.98, menunjukkan bahwa model mengenali sebagian besar data positif, meski masih terdapat sedikit prediksi positif palsu.
3. **Kelas Negatif**: Presisi dan recall sama-sama 0.00 karena ada keterbatasan dalam mengenali satu-satunya contoh negatif.
4. **Kelas Netral**: Recall rendah di 0.06, menunjukkan kesulitan model dalam mengenali sebagian besar data netral.

#### Prediksi Aspek

1. **Akurasi**: 0.5738 (sekitar 57.38%) – Model mampu memprediksi aspek dengan benar pada sekitar 57.38% data uji.
2. **Kelas Umum**: Memberikan performa tertinggi dengan F1-score 0.70, menunjukkan bahwa model cenderung lebih sering mengklasifikasikan aspek ke dalam kategori umum.
3. **Kelas Akademik**: Presisi rendah, menunjukkan bahwa data akademik sulit dikenali secara konsisten.
4. **Kelas Fasilitas**: Cenderung berada di antara akademik dan umum dalam presisi, tetapi recall 0.73 menunjukkan bahwa model dapat mengenali beberapa contoh fasilitas dengan baik.

### Evaluasi SVM-PSO IndoBERT pada Pembagian Data 80:20
#### Prediksi Sentimen

1. **Akurasi**: 0.6829 (sekitar 68.29%) – Akurasi sedikit menurun dibandingkan pembagian data 70:30.
2. **Kelas Positif**: Presisi 0.60 dan recall 1.00, menunjukkan kinerja yang baik dalam mengenali semua contoh positif tanpa kesalahan positif palsu.
3. **Kelas Netral**: Memiliki presisi dan recall rendah, dengan recall 0.00, menunjukkan model tidak mengenali data netral.

#### Prediksi Aspek

1. **Akurasi**: 0.4390 (sekitar 43.90%) – Akurasi cukup rendah untuk prediksi aspek.
2. **Kelas Umum**: Memberikan hasil terbaik dengan F1-score 0.61, menunjukkan kecenderungan model untuk mengenali kelas ini dibandingkan kelas lainnya.
3. **Kelas Akademik**: Rendah dalam presisi dan recall, menunjukkan model kesulitan mengenali aspek ini pada pembagian 80:20.
4. **Kelas Fasilitas**: Presisi dan recall juga rendah, mengindikasikan tantangan dalam mengenali data fasilitas dengan pembagian ini.

### Evaluasi SVM-PSO IndoBERT pada Pembagian Data 90:10
#### Prediksi Sentimen

1. **Akurasi**: 0.6190 (sekitar 61.90%) – Akurasi menurun lebih jauh dibandingkan dengan 70:30 dan 80:20.
2. **Kelas Positif**: Memiliki F1-score tertinggi sebesar 0.75, menunjukkan bahwa model tetap konsisten mengenali data positif meskipun akurasi keseluruhan lebih rendah.
3. **Kelas Netral**: Recall dan presisi tetap rendah dengan nilai 0.00, menunjukkan bahwa model masih memiliki keterbatasan mengenali kelas ini.

#### Prediksi Aspek

1. **Akurasi**: 0.5714 (sekitar 57.14%) – Meningkat sedikit dari pembagian data 80:20.
2. **Kelas Umum**: Tetap memiliki performa terbaik dengan F1-score 0.70, mengindikasikan kecenderungan model mengklasifikasikan banyak data ke kelas ini.
3. **Kelas Akademik**: F1-score 0.00 menunjukkan bahwa kelas akademik sulit dikenali.
4. **Kelas Fasilitas**: Presisi dan recall yang rendah menunjukkan bahwa model masih kesulitan mengenali data fasilitas dengan baik.

### Kesimpulan

1. **Performa SVM-PSO pada Sentimen dan Aspek**: Model SVM-PSO dengan representasi IndoBERT menunjukkan akurasi tertinggi pada pembagian data 70:30 untuk prediksi sentimen. Prediksi aspek cenderung menunjukkan performa rendah pada pembagian data 80:20, tetapi meningkat kembali pada pembagian 90:10. IndoBERT lebih konsisten pada kelas positif dalam prediksi sentimen, tetapi masih mengalami kesulitan dalam mengenali kelas netral dan akademik.

2. **Pengaruh Pembagian Data**: Pembagian data 70:30 memberikan hasil paling stabil, dengan akurasi yang lebih tinggi dalam prediksi sentimen. Penurunan performa terlihat pada pembagian data yang lebih kecil untuk pelatihan, terutama pada prediksi aspek dengan pembagian 80:20.

**Rekomendasi**: IndoBERT sebagai representasi fitur menunjukkan performa yang baik pada prediksi sentimen untuk kelas positif tetapi masih memerlukan peningkatan pada aspek dan kelas yang lebih sulit dikenali seperti netral dan akademik. Penulis dapat mempertimbangkan untuk menambah data atau menggunakan teknik augmentasi data untuk memperbaiki kinerja pada kelas-kelas minoritas ini.


# Pembahasan SVM-PSO TF-IDF vs Indo BERT

### Tabel Perbandingan Hasil Eksperimen SVM-PSO dengan TF-IDF vs IndoBERT

| Pembagian Data | Model           | Parameter Terbaik (C) | Parameter Terbaik (Gamma) | Akurasi Sentimen | Akurasi Aspek | F1-Score Sentimen Positif | F1-Score Aspek Umum |
|----------------|-----------------|------------------------|----------------------------|-------------------|---------------|---------------------------|----------------------|
| 7:3            | SVM-PSO TF-IDF  | 20.0                  | 1.0                        | 77.00%            | 61.00%        | 0.76                      | 0.86                 |
| 7:3            | SVM-PSO IndoBERT| 20.57                 | 1.09                       | 72.13%            | 57.38%        | 0.83                      | 0.70                 |
| 8:2            | SVM-PSO TF-IDF  | 17.0                  | 0.1                        | 78.00%            | 54.00%        | 0.85                      | 0.72                 |
| 8:2            | SVM-PSO IndoBERT| 0.29                  | 0.298                      | 68.29%            | 43.90%        | 0.81                      | 0.61                 |
| 9:1            | SVM-PSO TF-IDF  | 15.0                  | 0.05                       | 71.00%            | 62.00%        | 1.00                      | 0.73                 |
| 9:1            | SVM-PSO IndoBERT| 15.86                 | 0.0776                     | 61.90%            | 57.14%        | 0.75                      | 0.70                 |

### Interpretasi
- **Akurasi Sentimen**: SVM-PSO dengan representasi TF-IDF cenderung menghasilkan akurasi lebih tinggi pada prediksi sentimen dibandingkan dengan IndoBERT pada sebagian besar pembagian data, dengan akurasi tertinggi pada pembagian 80:20.
- **Akurasi Aspek**: Untuk prediksi aspek, TF-IDF juga sedikit lebih unggul pada beberapa pembagian data. Namun, pada pembagian data 90:10, IndoBERT menunjukkan peningkatan yang mendekati hasil TF-IDF.
- **F1-Score Sentimen Positif**: Kedua model mencapai F1-score yang tinggi pada kelas positif, dengan TF-IDF menunjukkan kinerja yang sedikit lebih konsisten di semua pembagian data.
- **F1-Score Aspek Umum**: IndoBERT memberikan hasil yang sedikit lebih rendah dibandingkan TF-IDF dalam mengenali kelas umum, meskipun masih mendekati dalam beberapa kasus.

**Kesimpulan**:
- **Pemilihan Representasi Fitur**: TF-IDF memberikan hasil yang lebih stabil dan cenderung lebih baik pada akurasi dan f1-score di sebagian besar pembagian data. IndoBERT mungkin memerlukan tuning tambahan atau data yang lebih besar untuk menunjukkan keunggulan representasinya pada tugas ini.
- **Pengaruh Pembagian Data**: Pembagian data 70:30 memberikan hasil paling seimbang di antara kedua model. Pembagian data yang lebih besar untuk pelatihan (80:20 atau 90:10) menunjukkan sedikit penurunan pada akurasi aspek dan sentimen, terutama untuk IndoBERT.



# PEMBAHASAN

### Hasil dan Pembahasan

Penelitian ini mengevaluasi performa model SVM-PSO yang dioptimalkan dengan dua pendekatan representasi fitur, yaitu **TF-IDF** dan **IndoBERT**, untuk memprediksi sentimen dan aspek dari data review. Tahapan penelitian meliputi proses pengolahan data, pemodelan dengan pengaturan parameter optimal, dan perbandingan hasil pada berbagai rasio pembagian data (70:30, 80:20, dan 90:10). Setiap tahap memiliki tujuan dan hasil yang berbeda, yang membantu memahami perbedaan kinerja model pada kedua representasi fitur tersebut.

#### 1. Proses Pengolahan Data

Pada tahap pengolahan data, beberapa langkah utama dilakukan. Pertama, proses tokenisasi diterapkan pada data untuk masing-masing representasi fitur. Pada pendekatan **TF-IDF**, data teks diubah menjadi vektor yang menggambarkan frekuensi term yang terstandarisasi pada tiap dokumen. Di sisi lain, pendekatan **IndoBERT** memanfaatkan representasi yang lebih mendalam dengan menggunakan pre-trained model IndoBERT untuk menghasilkan embedding teks yang lebih kaya informasi.

Setelah proses tokenisasi dan representasi fitur selesai, data dibagi menjadi beberapa skenario pembagian: **70:30, 80:20, dan 90:10** untuk data latih dan uji. Pembagian data dilakukan secara acak untuk memastikan distribusi yang konsisten antara data latih dan uji. Data yang digunakan dalam penelitian ini berlabel dua kategori, yaitu **sentimen** (positif, netral, negatif) dan **aspek** (akademik, fasilitas, umum).

#### 2. Modelling dengan SVM-PSO

Pada tahap pemodelan, model **Support Vector Machine (SVM)** dioptimalkan menggunakan **Particle Swarm Optimization (PSO)** untuk mencari parameter `C` dan `gamma` terbaik pada setiap skenario pembagian data, baik untuk TF-IDF maupun IndoBERT. Berikut adalah hasil dari parameter optimasi untuk kedua representasi fitur tersebut:

| Pembagian Data | Model           | Parameter Terbaik (C) | Parameter Terbaik (Gamma) | Akurasi Sentimen | Akurasi Aspek | F1-Score Sentimen Positif | F1-Score Aspek Umum |
|----------------|-----------------|------------------------|----------------------------|-------------------|---------------|---------------------------|----------------------|
| 70:30          | SVM-PSO TF-IDF  | 20.0                  | 1.0                        | 77.00%            | 61.00%        | 0.76                      | 0.86                 |
| 70:30          | SVM-PSO IndoBERT| 20.57                 | 1.09                       | 72.13%            | 57.38%        | 0.83                      | 0.70                 |
| 80:20          | SVM-PSO TF-IDF  | 17.0                  | 0.1                        | 78.00%            | 54.00%        | 0.85                      | 0.72                 |
| 80:20          | SVM-PSO IndoBERT| 0.29                  | 0.298                      | 68.29%            | 43.90%        | 0.81                      | 0.61                 |
| 90:10          | SVM-PSO TF-IDF  | 15.0                  | 0.05                       | 71.00%            | 62.00%        | 1.00                      | 0.73                 |
| 90:10          | SVM-PSO IndoBERT| 15.86                 | 0.0776                     | 61.90%            | 57.14%        | 0.75                      | 0.70                 |

#### 3. Evaluasi dan Perbandingan Hasil Eksperimen

Hasil evaluasi menunjukkan bahwa performa model SVM-PSO dengan TF-IDF umumnya lebih tinggi dibandingkan dengan IndoBERT dalam hal akurasi sentimen dan aspek. Pada rasio pembagian data **70:30**, SVM-PSO dengan TF-IDF mencapai akurasi sentimen tertinggi sebesar 77%, sedangkan akurasi dengan IndoBERT mencapai 72.13%. Selain itu, pada pembagian data yang sama, TF-IDF menunjukkan f1-score yang lebih tinggi untuk kategori aspek `umum`, yaitu 0.86, dibandingkan dengan IndoBERT yang hanya mencapai f1-score 0.70.

Untuk rasio pembagian **80:20**, model TF-IDF menunjukkan peningkatan performa dengan akurasi sentimen 78% dan akurasi aspek sebesar 54%. Sebaliknya, IndoBERT mencapai akurasi yang lebih rendah, yaitu 68.29% untuk sentimen dan 43.90% untuk aspek, mengindikasikan bahwa IndoBERT sedikit menurun dalam kinerja pada rasio ini.

Pada pembagian **90:10**, model TF-IDF dan IndoBERT menunjukkan perbedaan yang mencolok dalam f1-score pada sentimen positif, di mana TF-IDF memiliki skor 1.00, sedangkan IndoBERT hanya mencapai 0.75. Untuk aspek, kedua model memiliki hasil yang lebih seimbang, meskipun TF-IDF masih sedikit lebih unggul dengan f1-score 0.73 dibandingkan 0.70 untuk IndoBERT.

Secara keseluruhan, TF-IDF menunjukkan performa yang lebih stabil dan konsisten pada ketiga skenario pembagian data. Model IndoBERT memberikan representasi teks yang lebih kompleks, namun kinerjanya tidak selalu lebih baik daripada TF-IDF pada skenario pembagian data yang berbeda.

### Kesimpulan

Eksperimen ini menunjukkan bahwa representasi fitur yang berbeda memengaruhi performa prediksi sentimen dan aspek secara signifikan. **TF-IDF** memberikan akurasi dan f1-score yang lebih baik secara konsisten dibandingkan **IndoBERT**. Namun, dengan kompleksitas dan kedalaman representasi yang dihasilkan oleh IndoBERT, hasil ini menunjukkan potensi bagi IndoBERT dalam memproses data yang lebih besar atau lebih variatif. Oleh karena itu, penulis menyarankan untuk mempertimbangkan **IndoBERT** pada data yang memiliki banyak variasi makna dalam teks, sementara **TF-IDF** lebih disarankan untuk kasus dengan data yang lebih terstruktur dan seragam.
